In [ ]:
!git clone https://github.com/Guankai-sudo/uzi.git

Cloning into 'uzi'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), done.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/AI_studio
!pwd
!unzip archive.zip -d ./dataset

/content/gdrive/MyDrive/AI_studio
/content/gdrive/MyDrive/AI_studio
Archive:  archive.zip
  inflating: ./dataset/Img/img001-001.png  
  inflating: ./dataset/Img/img001-002.png  
  inflating: ./dataset/Img/img001-003.png  
  inflating: ./dataset/Img/img001-004.png  
  inflating: ./dataset/Img/img001-005.png  
  inflating: ./dataset/Img/img001-006.png  
  inflating: ./dataset/Img/img001-007.png  
  inflating: ./dataset/Img/img001-008.png  
  inflating: ./dataset/Img/img001-009.png  
  inflating: ./dataset/Img/img001-010.png  
  inflating: ./dataset/Img/img001-011.png  
  inflating: ./dataset/Img/img001-012.png  
  inflating: ./dataset/Img/img001-013.png  
  inflating: ./dataset/Img/img001-014.png  
  inflating: ./dataset/Img/img001-015.png  
  inflating: ./dataset/Img/img001-016.png  
  inflating: ./dataset/Img/img001-017.png  
  inflating: ./dataset/Img/img001-018.png  
  inflating: ./dataset/Img/img001-019.png  
  inflating: ./dataset/Img/img001-020.png  
  inflating: ./dataset/Img/img

# import libraries

In [2]:
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Data Overview

In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/AI_studio/dataset/english.csv')

In [4]:
df.head(10)

,image,label
0,Img/img001-001.png,0
1,Img/img001-002.png,0
2,Img/img001-003.png,0
3,Img/img001-004.png,0
4,Img/img001-005.png,0
5,Img/img001-006.png,0
6,Img/img001-007.png,0
7,Img/img001-008.png,0
8,Img/img001-009.png,0
9,Img/img001-010.png,0


it contain two columns:
- image -> path for the images
- label -> the digit or character of the images

In [5]:
df.shape

(3410, 2)

the dataset contains **``55 images``** for every character or digit

# load and preprocess images

In [6]:
img_size = (32, 32)

In [7]:
def load_and_preprocess_image(image_path):
    image_path = '/content/gdrive/MyDrive/AI_studio/dataset/' + image_path
    img = load_img(image_path, target_size=img_size, color_mode='grayscale')
    return img_to_array(img) / 255.0

In [8]:
image_paths = df['image'].values
labels = df['label'].values

In [9]:
# Encode character labels to integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# preprocess images

In [10]:
imgs = np.array([load_and_preprocess_image(img_path) for img_path in image_paths])

In [11]:
num_label = len(np.unique(labels_encoded))
one_hot_labels = np.eye(num_label)[labels_encoded]

# Split data

In [12]:
X_train, X_val, y_train, y_val = train_test_split(imgs, one_hot_labels, test_size=0.1, random_state=42)

# Data Augmentation

In [13]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(X_train)

# Modeling

In [14]:
# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_label, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

In [17]:
# Train the model with data augmentation
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    epochs=50,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


96/96 ━━━━━━━━━━━━━━━━━━━━ 12s 58ms/step - accuracy: 0.0382 - loss: 4.5873 - val_accuracy: 0.0147 - val_loss: 5.9214 - learning_rate: 0.0010
Epoch 2/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.1029 - loss: 3.6689 - val_accuracy: 0.0147 - val_loss: 6.8718 - learning_rate: 0.0010
Epoch 3/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.2080 - loss: 3.0503 - val_accuracy: 0.0293 - val_loss: 5.7443 - learning_rate: 0.0010
Epoch 4/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.3059 - loss: 2.5570 - val_accuracy: 0.1437 - val_loss: 3.7103 - learning_rate: 0.0010
Epoch 5/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.3962 - loss: 2.1513 - val_accuracy: 0.4457 - val_loss: 2.1885 - learning_rate: 0.0010
Epoch 6/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4636 - loss: 1.8701 - val_accuracy: 0.4428 - val_loss: 1.9938 - learning_rate: 0.0010
Epoch 7/50
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5187 - loss: 1.6428 - val_accuracy: 0.6

In [18]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7799 - loss: 0.6365 
Validation Loss: 0.5394537448883057
Validation Accuracy: 0.829912006855011
